In [ ]:
#import packages
import ast
import swifter
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets
import pandas as pd

In [ ]:
#notes:
#Blame and praise
# entailsments
#does not entail an actually entailment

#Blame vs endorsement
#Also report initial inspection of model wher praise and neutral was added to the hypothesis paramenter instead of blame/not blame.
# something about hieracivcal order and the words not being complete opisats and therefore the relative probabilities entails needed information
# in addition to the absolute probability

In [ ]:
#define functions


def extract_blame_from_paragraph_lookup(input_str):
    """
    Returns a binary list for blame per sentence:
    1 if blame is highest among labels and >= 0.8, else 0
    Handles arbitrary label order.
    """
    try:
        
        sentence_list = ast.literal_eval(input_str)
    except Exception:
        return []

    # List comprehension is faster than appending in a loop
    blame_binary = [
        int(
            (label_score := {label: score for label, score in zip(sent['labels'], sent['scores'])})['blame']
            >= max(label_score.get('praise', 0.0), label_score.get('neutral', 0.0), 0.8)
        )
        for sent in sentence_list
    ]


    return blame_binary

#example usage: final_data['blame_binary'] = final_data['blame_in_text'].swifter.apply(extract_blame_from_paragraph_lookup)


#get row indices (paragraphs) that contain blame
def get_rows_with_blame(df, col="blame_binary"):
    """
    Returns row indices where the list in `col` contains at least one 1.
    Handles both real lists and stringified lists.
    """
    indices = []
    for i, values in zip(df.index, df[col]):
        if isinstance(values, str):  # convert only if it's a string
            values = ast.literal_eval(values)
        if 1 in values:
            indices.append(i)
    return indices

#example usage: row_indices = get_rows_with_blame(final_data, col="blame_binary")
#print(row_indices[:10])


#get paragraphs and sentence indices of blame True
def get_rows_and_positions(df, col="blame_binary"):
    """
    Returns {row_index: [positions_of_1s]}.
    Handles both real lists and stringified lists.
    """
    results = {}
    for i, values in zip(df.index, df[col]):
        if isinstance(values, str):
            values = ast.literal_eval(values)
        ones = [j for j, v in enumerate(values) if v == 1]
        if ones:
            results[i] = ones
    return results

#example usage: rows_with_positions = get_rows_and_positions(final_data, col="blame_binary")
#print(rows_with_positions)

#
#get danish sentences containing blame from indices extracted as above

def danish_sentences_with_blame_extraction(dict, data, text_column):


    rows = list(dict.keys())
    sentences = {}

    for para in rows:
        sentence_indices = dict[para]
        text_sentences = ast.literal_eval(data.loc[para][text_column])
        
        blame_sentence_dict = {}
        for indx in sentence_indices:
            blame_sentence = text_sentences[indx]
            blame_sentence_dict[indx] = blame_sentence
        
        
        sentences[para] = blame_sentence_dict

    return sentences


#example usage: danish_sentences_with_blame = danish_sentences_with_blame_extraction(rows_with_positions, final_data, 'da_segmented_text')


#extract blame percentage:

import ast

def total_blame_percentage(string_rows):
    """
    Calculate the total percentage of blame-sentences across all rows,
    converting string representations of lists into actual lists.
    
    Parameters
    ----------
    string_rows : list of str
        Each element is a string like '[1, 0, 1]' representing a row of blame labels.

    Returns
    -------
    float
        Total percentage of blame-sentences (0–100).
    """
    total_sentences = 0
    total_blame = 0

    for row_str in string_rows:
        try:
            row = ast.literal_eval(row_str)  # convert string to list
            if not isinstance(row, list):
                continue  # skip if not a list
            row = [int(val) for val in row]  # ensure integers
            total_sentences += len(row)
            total_blame += sum(row)
        except (ValueError, SyntaxError):
            continue  # skip invalid rows

    if total_sentences == 0:
        return 0.0

    return (total_blame / total_sentences) * 100, total_sentences, total_blame

#Example usage: percentage_blame, total_sent, total_blame= total_blame_percentage(final_data['blame_binary'])

#extract blame scores

def get_blame_scores(data = final_data, blame_in_text_column = 'blame_in_text'):
    all_blame_scores = []

    for i in range(len(data)):
        dict_labels = ast.literal_eval(final_data.loc[i][blame_in_text_column])

        blame_list = [(label_score := {label: score for label, score in zip(sent['labels'], sent['scores'])})['blame'] for sent in dict_labels]

        all_blame_scores +=blame_list
    return all_blame_scores


#make vizualization of the distribution

def vizualize_blame_prob(blame_scores):
    plt.figure()
    plt.hist(blame_scores, log = True, bins = 50)
    plt.title('Distribution of blame probabilities in training data (log transformed counts)')
    plt.xlabel('Probability of blame')
    plt.ylabel('Log Counts')
    plt.show()

    plt.figure()
    plt.hist(blame_scores, bins = 50)
    plt.title('Distribution of blame probabilities in training data')
    plt.xlabel('Probability of blame')

    plt.ylabel('Absolute Counts')
    plt.show()


In [ ]:
#Sanity checks:
#check if translated, orignal and blame probabilty are same lengt (extend for all templates)
for indx in range(len(final_data)):

    temo = final_data.loc[indx]

    l_da = len(ast.literal_eval(temo['da_segmented_text']))
    l_en = len(ast.literal_eval(temo['translated_text']))
    l_bl = len(ast.literal_eval(temo['blame_in_text']))

    if l_da != l_en != l_bl:
        print(temo)

In [ ]:
#Data with all templates


all_templates_data = pd.read_csv("/work/MarkusLundsfrydJensen#1865/Bachelor_project/annotation_data_fifth_template_appended.csv")

all_templates_data.head()

In [ ]:


columns = ["blame_in_text","second_template_blame_in_text","third_template_blame_in_text","fourth_template_blame_in_text","fifth_template_blame_in_text"]

for i, column in enumerate(columns):
    all_templates_data[f'blame_binary_temp_{i+1}'] = all_templates_data[column].swifter.apply(extract_blame_from_paragraph_lookup)

all_templates_data.head()